# OCR with llama:vision

In [1]:
import ollama
import os

In [2]:
image_path = os.path.join(os.getcwd(), "pics", "invoice-template.webp") # Replace with your image path

# Use Ollama to analyze the image with Llama 3.2-Vision
response = ollama.chat(
    model="llama3.2-vision",
    messages=[{
      "role": "user",
      "content": "Analyze the text in the provided image. Extract all readable content and present it in a structured [Markdown/JSON] format.",
      "images": [image_path]
    }],
)

In [3]:
# Extract the model's response about the image
cleaned_text = response['message']['content'].strip()
print(f"Model Response: {cleaned_text}")

Model Response: **Invoice**

* **Invoice Number**: 01234
* **Date**: 11.02.2030
* **Due Date**: 11.03.2030

**Payment Information**

* **Payee**: Borcele Bank
* **Account Name**: Adeline Palmerston
* **Account Number**: 0123 4567 8901

**Invoice Details**

* **Invoice Total**: $400
* **Tax**: 10%
* **Total**: $440

**Services Provided**

* **Brand Consultation**: $100
* **Logo Design**: $100
* **Website Design**: $100
* **Social Media Templates**: $100
* **Brand Photography**: $100
* **Brand Guide**: $100

**Subtotal**: $400
**Tax**: $40
**Total**: $440

**Payment Method**

* **Payment Method**: Bank Transfer
* **Payment Date**: 11.03.2030

**Signature**

* **Signature**: Adeline Palmerston


# Structured JSOn Output

In [4]:


from langchain_ollama import ChatOllama
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re



In [5]:
# Define the template
def get_response(response):
    template = """
        You are given text that may contain receipt or invoice information.  
        Your task is to extract structured data and return it strictly as JSON.

        Fields to extract:
        - Company
        - Bill To: {{ Name, Address }}
        - Ship To: {{ Name, Address }}
        - Receipt Number
        - Date
        - P.O. Number
        - Items: [{{ Description, Quantity, Unit Price, Total }}]
        - Subtotal
        - Sales Tax
        - Total
        - Payment Instructions: {{ Bank Transfer, Paypal Email }}
        - Terms & Conditions

        Input text:
        {response}

        Return only valid JSON (UTF-8 encoded), without any extra commentary or Markdown formatting.
            """
    
#
    prompt = ChatPromptTemplate.from_template(template)

    # llm
    llm = ChatOllama(model="llama3",  temperature=0)

    chain = ( prompt
    | llm
    | StrOutputParser()
    )

    return chain.invoke({
    "response": response,
    })


In [6]:
result = get_response(cleaned_text)

In [7]:
result

'Here is the extracted data in JSON format:\n\n```\n{\n    "Company": "",\n    "Bill To": {\n        "Name": "",\n        "Address": ""\n    },\n    "Ship To": {\n        "Name": "",\n        "Address": ""\n    },\n    "Receipt Number": "01234",\n    "Date": "11.02.2030",\n    "P.O. Number": "",\n    "Items": [\n        {\n            "Description": "Brand Consultation",\n            "Quantity": 1,\n            "Unit Price": 100,\n            "Total": 100\n        },\n        {\n            "Description": "Logo Design",\n            "Quantity": 1,\n            "Unit Price": 100,\n            "Total": 100\n        },\n        {\n            "Description": "Website Design",\n            "Quantity": 1,\n            "Unit Price": 100,\n            "Total": 100\n        },\n        {\n            "Description": "Social Media Templates",\n            "Quantity": 1,\n            "Unit Price": 100,\n            "Total": 100\n        },\n        {\n            "Description": "Brand Photography"

In [8]:
import re
import json

json_match = re.search(r"```\n(.*?)\n```", result, re.DOTALL)

if json_match:
    receipt_data = json_match.group(1)
    try:
        parsed_data = json.loads(receipt_data)
        json_data = json.dumps(parsed_data, indent=2)
        print(json.dumps(parsed_data, indent=2))
    except json.JSONDecodeError:
        print("Failed to parse JSON.")
else:
    print("No JSON found.")


{
  "Company": "",
  "Bill To": {
    "Name": "",
    "Address": ""
  },
  "Ship To": {
    "Name": "",
    "Address": ""
  },
  "Receipt Number": "01234",
  "Date": "11.02.2030",
  "P.O. Number": "",
  "Items": [
    {
      "Description": "Brand Consultation",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    },
    {
      "Description": "Logo Design",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    },
    {
      "Description": "Website Design",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    },
    {
      "Description": "Social Media Templates",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    },
    {
      "Description": "Brand Photography",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    },
    {
      "Description": "Brand Guide",
      "Quantity": 1,
      "Unit Price": 100,
      "Total": 100
    }
  ],
  "Subtotal": 400,
  "Sales Tax": 40,
  "Total": 440,
  "Payment Instru